### Create zarr stores

This notebook is used for creating zarr stores from large collections of netcdf files. 
Due to the large memory use when using raw netcdf outputs (croco outputs could be 1GB per month without problems), i've found that transforming all the dataset to a zarr store as the first postprocess task it will highly increase the performance of the next operations, especially when using dask for parallel computing. This notebook is heavily based on the Raphael Dussing article about modern tools for large ROMS simulations https://raphaeldussin.medium.com/modern-python-tools-for-the-roms-ocean-model-bfca8642db01

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import xarray as xr
import pandas as pd
import numpy as np
import datetime

from glob import glob
import sys
sys.path.append('../')
from utils import *
from load import *
from numerics import *

from dask.distributed import LocalCluster, Client
import gc
cluster = LocalCluster(n_workers=20, threads_per_worker=1, memory_limit='auto')
client  = Client(cluster,asynchronous=True)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 20
Total threads: 20,Total memory: 52.43 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39629,Workers: 20
Dashboard: http://127.0.0.1:8787/status,Total threads: 20
Started: Just now,Total memory: 52.43 GiB
Comm: tcp://127.0.0.1:38336,Total threads: 1
Dashboard: http://127.0.0.1:42829/status,Memory: 2.62 GiB
Nanny: tcp://127.0.0.1:36379,


---
### RUND0

In [4]:
#Load crocod0 grid
gridd0 = xr.open_dataset('data/DESALADORAS_RUND0/crocod0_grd.nc')
gridd0 = gridd0[['x_rho','y_rho','lon_rho','lat_rho','h','hraw','f','mask_rho']].squeeze()
gridd0

<xarray.Dataset>
Dimensions:   (eta_rho: 252, xi_rho: 152)
Dimensions without coordinates: eta_rho, xi_rho
Data variables:
    x_rho     (eta_rho, xi_rho) float64 ...
    y_rho     (eta_rho, xi_rho) float64 ...
    lon_rho   (eta_rho, xi_rho) float64 ...
    lat_rho   (eta_rho, xi_rho) float64 ...
    h         (eta_rho, xi_rho) float64 ...
    hraw      (eta_rho, xi_rho) float64 ...
    f         (eta_rho, xi_rho) float64 ...
    mask_rho  (eta_rho, xi_rho) float64 ...
Attributes:
    title:    DESALADORAS_D0
    date:     08-Mar-2023
    type:     CROCO grid file

In [26]:
gridd0.to_zarr('data/ZARR/simsequia_rund0_control', consolidated=True, mode='w')
gridd0.to_zarr('data/ZARR/simsequia_rund0_rivers', consolidated=True, mode='w')

In [29]:
#Load croco simulations
paths_control   = sorted(glob('SIMSEQUIA/data/DESALADORAS_RUND0/*avg*'))[24:]
paths_rivers    = sorted(glob('SIMSEQUIA/data/DESALADORAS_RUND0_RIOS/*avg*'))

for v in ['hbl']:
    print('Processing variable: ',v)
    control = []
    rivers  = []
    for pc,pr in zip(paths_control, paths_rivers):
        print('control: '+pc.split('_')[-1],'rivers: '+pr.split('_')[-1], sep='\t')
        datac = load_croco(pc, YORIG='1949-12-31 00:00:00', chunks={'time':1}) #load_croco -> load.py
        datar = load_croco(pr, YORIG='1949-12-31 00:00:00', chunks={'time':1})
        
        datac = datac[v]
        datar = datar[v]
    
        control.append(datac)
        rivers.append(datar)
        del datac,datar
        client.run(trim_memory) # Check utils.py
    print('Joining...')
    control = xr.concat(control, 'time').sortby('time').chunk({'time':1, 'eta_rho':-1, 'xi_rho':-1})
    rivers  = xr.concat(rivers, 'time').sortby('time').chunk({'time':1, 'eta_rho':-1, 'xi_rho':-1})
    print('Saving to disk...')
    print('control')
    control.to_dataset(name=v).to_zarr('SIMSEQUIA/data/ZARR/simsequia_rund0_control', consolidated=True, mode='a')
    client.run(trim_memory)
    print('rivers')
    rivers.to_dataset(name=v).to_zarr('SIMSEQUIA/data/ZARR/simsequia_rund0_rivers', consolidated=True, mode='a')
    client.run(trim_memory)
    del control,rivers
    print('Done')
    

Processing variable:  hbl
control: Y2001M1.nc	rivers: Y2001M1.nc
control: Y2001M10.nc	rivers: Y2001M10.nc
control: Y2001M11.nc	rivers: Y2001M11.nc
control: Y2001M12.nc	rivers: Y2001M12.nc
control: Y2001M2.nc	rivers: Y2001M2.nc
control: Y2001M3.nc	rivers: Y2001M3.nc
control: Y2001M4.nc	rivers: Y2001M4.nc
control: Y2001M5.nc	rivers: Y2001M5.nc
control: Y2001M6.nc	rivers: Y2001M6.nc
control: Y2001M7.nc	rivers: Y2001M7.nc
control: Y2001M8.nc	rivers: Y2001M8.nc
control: Y2001M9.nc	rivers: Y2001M9.nc
control: Y2002M1.nc	rivers: Y2002M1.nc
control: Y2002M10.nc	rivers: Y2002M10.nc
control: Y2002M11.nc	rivers: Y2002M11.nc
control: Y2002M12.nc	rivers: Y2002M12.nc
control: Y2002M2.nc	rivers: Y2002M2.nc
control: Y2002M3.nc	rivers: Y2002M3.nc
control: Y2002M4.nc	rivers: Y2002M4.nc
control: Y2002M5.nc	rivers: Y2002M5.nc
control: Y2002M6.nc	rivers: Y2002M6.nc
control: Y2002M7.nc	rivers: Y2002M7.nc
control: Y2002M8.nc	rivers: Y2002M8.nc
control: Y2002M9.nc	rivers: Y2002M9.nc
control: Y2003M1.nc	rivers